## This notebook is created to understand how Q-Learning works. 
Find more information about Gym and Taxi-v2:

https://gym.openai.com/envs/Taxi-v2/

https://github.com/openai/gym/blob/master/gym/envs/toy_text/taxi.py

In [1]:
import numpy as np
import gym
import random

In [2]:
# Description:
#    There are four designated locations in the grid world indicated by R(ed), B(lue), G(reen), and Y(ellow). 
#    When the episode starts, the taxi starts off at a random square and the passenger is at a random location. 
#    The taxi drive to the passenger's location, pick up the passenger, drive to the passenger's destination 
#    (another one of the four specified locations), and then drop off the passenger. Once the passenger is dropped
#    off, the episode ends.

env = gym.make("Taxi-v2")
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [3]:
# Observations: 
# There are 500 discrete states since there are 25 taxi positions, 5 possible locations of the passenger 
# (including the case when the passenger is the taxi), and 4 destination locations. 
action_size = env.action_space.n
print("Action size", action_size)

# Actions: 
#    There are 6 discrete deterministic actions:
#    - 0: move south
#    - 1: move north
#    - 2: move east 
#    - 3: move west 
#    - 4: pickup passenger
#    - 5: dropoff passenger
state_size = env.observation_space.n
print("State size", state_size)

Action size 6
State size 500


In [4]:
qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [5]:
total_episodes = 50000          # Total episodes
total_test_episodes = 100        # Total test episodes
max_steps = 99                   # Max steps per episodes
learning_rate = 0.8              # Learning rate
gamma = 0.95                     # Discounting rate

# Exploration parameters
epsilon = 1.0                    # Exploration rate
max_epsilon = 1.0                # Exploration probability at start
min_epsilon = 0.01               # Minimum exploration probability
decay_rate = 0.005               # Exponential decay rate for exploration probability
state = env.reset()

In [6]:
# For life or unitil learning is stopped
#     Choose an action (a) in the current world state (s) based on current Q-value estimates(Q(s,`))
#     Take the action (a) and observe the outcome state (s') and reward (r)
#     Update Q(s,a):=Q(s,a)+a[r+gamma*max(Q(s',a'))-Q(s,a)]

#  Rewards: 
#    There is a reward of -1 for each action and an additional reward of +20 for delievering the passenger. 
#    There is a reward of -10 for executing actions "pickup" and "dropoff" illegally.

rewards = []                     # List of rewards

# Run forever or till learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    
    # Choose an action a in the current world state (s)
    for step in range(max_steps):
        # First randomize a number to exploition/exploration trade off
        exp_tradeoff = random.uniform(0,1)
        
        # If the number > epsilon: exploition
        if exp_tradeoff > epsilon:
            # Return the index of the largest Q-value in the current state
            action = np.argmax(qtable[state,:])
        
        # Else doing a random choice: exploration
        else:
            action = env.action_space.sample()
        
        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)
        
        # Update Q(s,a) = Q(s,a) + lr*[R(s,a)+gamma*max Q(s',a')-Q(s,a)]
        # qtable[new_state,:]: all the actions we can take from new state
        qtable[state,action] = qtable[state, action] +learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        
        total_rewards += reward
        state = new_state
        
        # If done: finish episode
        if done == True:
            break
            
    # Reduce epsilon since we need more exploition and less exploration
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)
    rewards.append(total_rewards)

print("Average score over time:"+ str(sum(rewards)/total_episodes))
print(qtable)

Average score over time:6.50444
[[  0.           0.           0.           0.           0.
    0.        ]
 [218.89601556 258.63652618 244.69129616 258.55206877 273.30166436
  249.63655233]
 [272.85668342 288.70234325 273.30163454 288.73859191 304.98799375
  279.30599776]
 ...
 [ -3.23842335 322.09262437 243.43610511  -2.9374976   -8.98231217
  -12.04322304]
 [ -4.99782378  -5.0498312   -5.57220842 258.63655958 -12.36071629
  -12.86956401]
 [ -0.96       340.09749914  -1.568       -0.8        216.4862979
  -10.528     ]]


In [7]:
# Use the "cheatsheet" generated by the 50000 episodes training to see the agent run Taxi

env.reset()
rewards = []

for episode in range(total_test_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    print("---------------------------------------------")
    print("EPISODE", episode)
    
    for step in range(max_steps):
        env.render()
        # Take the actions with the highest expected Q-value: exploition
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, infor = env.step(action)
        
        total_rewards += reward
        state = new_state
        
        if done == True:
            rewards.append(total_rewards)
            break
    print("The score for this episode is", total_rewards)
env.close()
print("---------------------------------------------")
print("Score over time:" + str(sum(rewards)/total_test_episodes))        

---------------------------------------------
EPISODE 0
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
|_: : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
|_: : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| :_: : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : :_: : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R:

  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : |_: |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : :_: |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : :_: : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| :_: : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
|_: : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
|_| : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
The score for this episode is 7
---------------------------------------------
EPISODE 26
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : 

  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
The score for this episode is 6
---------------------------------------------
EPISODE 40
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B

### Conclusion
From the test running episodes, one can easily find that the agent works pretty well with the previous 50,000 episodes training. However, one issue raised here is that in some episodes the taxi doesn't go to the nearest passenger to pick up. Instead, the taxi goes to further passenger and drop the passenger to a further but legal spot. So even the taxi will receive +20 rewards when pick up and drop off successfully, it takes too many steps to make it. So further improvement could be step less and finally get more final rewards. More training may help with it.  